# IEEE14bus using DPsimLauncher

## Case 1: Different step sizes

In [ ]:
import sys
sys.path.insert(0,'/home/mmo/git/Can/dpsim/python/src/dpsim')
sys.path.insert(0,'/home/mmo/git/Can/dpsim/build')

import dpsimpy
from DPsimLauncher import DPsimLauncher

In [ ]:
launcher = DPsimLauncher('/home/mmo/git/Can/dpsim/python/src/dpsim/Config_IEEE14_ShortCircuit.json')
launcher.start_pf_simulation()
launcher.get_pf_results()

In [ ]:
launcher.run_dynamic_simulations()

In [ ]:
launcher.system

### Plot results

In [ ]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import matplotlib.pyplot as plt

var_name = 'Bus_0014.V'
timestep = 1e-6

ts_ref_simulation = None
for elem in launcher.get_path_to_results():
    if (elem["TimeStep"]==1e-4):
        for idx, domain in enumerate(elem["Domain"]):
            if domain=="EMT":
                ts_ref_simulation = read_timeseries_csv(elem["Path"][idx], print_status=False)
        
for elem in launcher.get_path_to_results():
    plt.figure()
    plt.title('Time step = {}s'.format(elem["TimeStep"]))
    plt.xlabel("Time [s]")
    plt.ylabel("Voltage [kV]")
    plt.xlim([0,0.2])
    plt.plot(ts_ref_simulation[var_name+"_0"].interpolate(timestep).time, ts_ref_simulation[var_name+"_0"].interpolate(timestep).values/1000, label='REF')
    for idx, domain in enumerate(elem["Domain"]):
        ts_dpsim = read_timeseries_csv(elem["Path"][idx], print_status=False)
        if domain=="EMT":
            #plt.plot(ts_dpsim[var_name+"_0"].interpolate(timestep).time, ts_dpsim[var_name+"_0"].interpolate(timestep).values/1000, label='{}'.format(domain))
            pass
        else:
            plt.plot(ts_dpsim[var_name].interpolate(timestep).time, np.sqrt(2/3)*ts_dpsim[var_name].interpolate(timestep).frequency_shift(60).values/1000, '--', label='{} backshift'.format(domain))
    plt.legend(loc="upper right")
    plt.grid()


### Export to CIM

In [ ]:
import cimpy
from cimpy.DPsim2CIM import DPsimToCIMpy

cim_topo = DPsimToCIMpy(launcher.system_pf, launcher.sim, launcher.system, frequency=60)
activeProfileList = ['SV', 'EQ', 'TP', 'DY']
cimpy.cim_export(cim_topo, 'test3', 'cgmes_v2_4_15', activeProfileList)